# Description

It standardizes the features (latent variables) of an input file.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from IPython.display import display

import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

import conf

# Settings

In [3]:
INPUT_FILEPATH = Path(
    conf.RESULTS['PROJECTIONS_DIR'],
    'projection-smultixcan-efo_partial-mashr-zscores.pkl'
).resolve()
display(INPUT_FILEPATH)

input_filepath_stem = INPUT_FILEPATH.stem
display(input_filepath_stem)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/projections/projection-smultixcan-efo_partial-mashr-zscores.pkl')

'projection-smultixcan-efo_partial-mashr-zscores'

In [4]:
# output dir for this notebook
RESULTS_DIR = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    'z_score_std'
).resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/z_score_std')

# Load input file

In [5]:
data = pd.read_pickle(INPUT_FILEPATH)

In [6]:
data.shape

(987, 3749)

In [7]:
data.head()

,100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,100004_raw-Fat,100005_raw-Carbohydrate,100006_raw-Saturated_fat,100007_raw-Polyunsaturated_fat,100008_raw-Total_sugars,100009_raw-Englyst_dietary_fibre,100010-Portion_size,...,visual impairment,vitiligo,vitreous body disease,vocal cord polyp,voice disorders,wellbeing measurement AND family relationship,wheezing,whooping cough,worry measurement,wrist fracture
LV1,-0.018452,-0.043782,-0.021514,-0.030454,-0.017428,-0.012313,-0.018044,-0.008047,-0.049581,-0.033719,...,-0.006604,-0.003207,-0.010638,-0.005853,0.001435,-0.013369,-0.005603,0.005034,0.045065,0.040257
LV2,0.052938,-0.012041,-0.028537,-0.052542,0.003757,-0.054680,-0.032025,0.009933,-0.030161,0.006869,...,-0.030526,-0.033616,0.018583,0.004988,-0.013814,0.052914,0.034170,-0.032019,-0.013778,0.022792
LV3,-0.003629,-0.011772,0.009441,0.000459,-0.003708,0.000021,-0.001102,-0.013368,-0.024807,-0.020284,...,-0.000083,-0.022389,-0.019574,-0.045773,0.006880,0.007325,0.048046,0.030989,0.088343,0.021850
LV4,0.028359,-0.006148,0.007808,-0.039613,-0.000929,-0.039796,-0.043357,0.007231,-0.002575,-0.003986,...,-0.018537,0.010687,-0.043556,-0.030884,-0.037816,0.043915,0.025911,-0.047740,-0.006550,0.054932
LV5,-0.015500,0.007011,0.012707,0.006191,-0.000647,0.032319,-0.028891,-0.002337,0.029445,0.008233,...,0.023084,-0.023192,0.010425,-0.006992,0.010299,-0.015184,-0.019313,-0.007507,-0.026180,0.049838


# Data preprocessing for clustering

In [8]:
data_stats = data.T.iloc[:, :10].describe()
display(data_stats)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10
count,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03
mean,1.184554e-18,-7.818055e-18,4.975126e-18,-7.699600e-19,-2.369108e-19,-1.125326e-18,-1.658375e-18,-9.476430e-19,4.501304e-18,3.316751e-18
std,3.332670e-02,2.860283e-02,3.791407e-02,3.044476e-02,3.008916e-02,2.615014e-02,3.737188e-02,3.153450e-02,3.179361e-02,2.975571e-02
min,-1.829498e-01,-9.410261e-02,-8.970296e-02,-2.330088e-01,-8.127525e-02,-2.021169e-01,-8.845736e-02,-8.158941e-02,-8.224399e-02,-1.225250e-01
25%,-2.004467e-02,-1.854991e-02,-2.237111e-02,-1.877114e-02,-1.979890e-02,-1.670325e-02,-2.203061e-02,-1.942657e-02,-1.838957e-02,-1.980324e-02
50%,-2.603228e-03,-1.359580e-03,-5.313336e-03,-2.637242e-04,-2.225820e-03,-1.166629e-04,-4.656835e-03,-3.405316e-03,-3.455237e-03,-1.833973e-03
75%,1.645519e-02,1.670710e-02,1.344975e-02,1.793625e-02,1.509677e-02,1.648301e-02,1.536048e-02,1.436112e-02,1.348235e-02,1.782587e-02
max,3.179871e-01,2.425872e-01,3.298316e-01,1.857484e-01,3.089461e-01,1.050431e-01,3.650229e-01,2.703459e-01,3.605632e-01,2.193655e-01


## Standardize

In [9]:
data_t = data.T

scaled_data = pd.DataFrame(
    data=scale(data_t),
    index=data_t.index.copy(),
    columns=data_t.columns.copy()
)

In [10]:
scaled_data.shape

(3749, 987)

In [11]:
scaled_data.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
100001_raw-Food_weight,-0.553743,1.851060,-0.095721,0.931603,-0.515196,1.371264,-0.413165,0.729668,0.546338,0.901153,...,1.148776,1.649018,-1.375952,2.203063,0.810199,-0.987048,0.277710,-0.607613,0.572376,-0.660846
100002_raw-Energy,-1.313905,-0.421015,-0.310524,-0.201978,0.233028,0.693838,0.084129,0.572439,0.217866,1.296973,...,0.165430,0.799276,-0.361989,-0.008253,2.063780,-0.930765,-0.147007,1.601938,-0.273553,-0.560822
100003_raw-Protein,-0.645622,-0.997845,0.249039,0.256499,0.422357,0.829189,-0.169004,0.511550,0.002389,-0.057200,...,1.016791,1.019488,-2.148844,-0.065130,0.041052,-0.836118,-0.320571,1.325519,-0.341494,-0.495786
100004_raw-Fat,-0.913924,-1.837183,0.012095,-1.301315,0.205794,1.129114,0.019985,0.376660,-0.810272,0.843611,...,0.544266,0.576714,0.281731,-0.218963,1.258957,-1.131924,0.378545,0.842047,-0.622194,-0.160058
100005_raw-Carbohydrate,-0.523004,0.131379,-0.097802,-0.030526,-0.021500,-0.219116,0.668229,0.365700,1.102347,0.845625,...,-0.344740,0.079274,0.253427,0.236474,1.033895,-0.229105,0.364793,1.776714,0.190531,-1.459641


In [12]:
scaled_data_stats = scaled_data.iloc[:,:10].describe()
display(scaled_data_stats)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10
count,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3749.000000,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03
mean,1.516229e-17,-7.581144e-18,3.790572e-18,-1.516229e-17,0.000000,-1.990050e-17,-3.790572e-17,-1.895286e-17,7.581144e-18,1.895286e-17
std,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00
min,-5.490319e+00,-3.290415e+00,-2.366270e+00,-7.654516e+00,-2.701507,-7.730123e+00,-2.367266e+00,-2.587652e+00,-2.587154e+00,-4.118246e+00
25%,-6.015400e-01,-6.486207e-01,-5.901263e-01,-6.166463e-01,-0.658095,-6.388294e-01,-5.895756e-01,-6.161241e-01,-5.784817e-01,-6.656160e-01
50%,-7.812280e-02,-4.753941e-02,-1.401602e-01,-8.663541e-03,-0.073984,-4.461865e-03,-1.246246e-01,-1.080014e-01,-1.086916e-01,-6.164252e-02
75%,4.938200e-01,5.841844e-01,3.547903e-01,5.892194e-01,0.501801,6.304060e-01,4.110718e-01,4.554706e-01,4.241150e-01,5.991536e-01
max,9.542784e+00,8.482362e+00,8.700611e+00,6.101977e+00,10.269057,4.017459e+00,9.768617e+00,8.574164e+00,1.134226e+01,7.373197e+00


In [13]:
assert np.all([np.isclose(scaled_data_stats.loc['mean', c], 0.0) for c in scaled_data_stats.columns])

In [14]:
assert np.all([np.isclose(scaled_data_stats.loc['std', c], 1.0, atol=1e-03) for c in scaled_data_stats.columns])

# Save

In [15]:
output_file = Path(
    RESULTS_DIR,
    f'z_score_standardized-{input_filepath_stem}.pkl',
).resolve()

display(output_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/z_score_std/z_score_standardized-projection-smultixcan-efo_partial-mashr-zscores.pkl')

In [16]:
scaled_data.to_pickle(output_file)